In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import h5py
import itertools

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from IPython import display
import os
from PIL import Image
from torch.utils.data.dataset import Dataset
from matplotlib.pyplot import imread
import glob
import os
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder

import torch.nn as nn
from torchvision.models import alexnet, vgg16, resnet152, resnet18, vgg19

%matplotlib inline

import matplotlib.pyplot as plt
import math

import torch.nn.functional as nnf
from torch.utils.data import random_split
from torch.optim import SGD 
from torch.distributions import constraints
import torchvision as torchv
import torchvision.transforms as torchvt
import pickle

from sklearn.metrics import classification_report, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import time

In [2]:
# res_net = resnet18(pretrained=True)
# # First we freeze all layers
# for param in res_net.parameters():
#     param.requires_grad = False # frozen layer

class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

# res_net.fc = Identity()
# res_net.cuda()

In [76]:
ckpt_gaf = torch.load('res_net_test_recall_best_gaf.chk')
res_net_gaf = resnet18(pretrained=True)
for param in res_net_gaf.parameters():
    param.requires_grad = False

num_ftrs = res_net_gaf.fc.in_features
res_net_gaf.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_gaf.load_state_dict(ckpt_gaf['net'])
res_net_gaf.fc = Identity()
res_net_gaf.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [4]:
ckpt_rp = torch.load('res_net_test_recall_best_rp.chk')
res_net_rp = resnet18(pretrained=True)
for param in res_net_rp.parameters():
    param.requires_grad = False

num_ftrs = res_net_rp.fc.in_features
res_net_rp.fc = nn.Sequential(
                nn.Linear(in_features=num_ftrs, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
                nn.Linear(in_features=128, out_features=5, bias=True))
res_net_rp.load_state_dict(ckpt_rp['net'])
res_net_rp.fc = Identity()
res_net_rp.cuda()   

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [3]:
# Function for moving tensor or model to GPU
def cuda(xs):
    if torch.cuda.is_available():
        if not isinstance(xs, (list, tuple)):
            return xs.cuda()
        else:
            return [x.cuda() for x in xs]
    else:
        return xs

# Custom class for defining dataset for training with augmentation
class Dataset_Hdf5(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
            # no augmentation as we are transforming heartbeats to images via gramian angular field
            self.transform = transforms.Compose([
#                                               transforms.ToPILImage(),
#                                               transforms.RandomRotation((0, 360)),
#                                               transforms.RandomHorizontalFlip(),
#                                               transforms.RandomVerticalFlip(),
                                              transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return self.transform(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [4]:
hb_train_loader_bln_3 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_mit_bih_balanced_3.hdf5', 'train'), 
                                                batch_size=512, shuffle=True)
hb_test_loader_bln_3 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_mit_bih_balanced_3.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [5]:
hb_train_loader_bln_4 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_mit_bih_balanced_rp.hdf5', 'train'), 
                                                batch_size=512, shuffle=False)
hb_test_loader_bln_4 = torch.utils.data.DataLoader(Dataset_Hdf5('/home/asif/heartbeat/hb_data_mit_bih_balanced_rp.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [17]:
train_size = 0
for i, data in enumerate(hb_train_loader_bln_3, 0):
    train_size += data[0].shape[0]

In [13]:
test_size = 0
for i, data in enumerate(hb_test_loader_bln_3, 0):
    test_size += data[0].shape[0]

In [6]:
train_size = 152471
test_size = 21892

In [10]:
train_size, test_size

(152471, 21892)

In [10]:
with h5py.File('hb_gaf_bih_intermediate.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (train_size, 512), np.float32)
    hdf5_file.create_dataset("y_train", (train_size,), np.int32)
    hdf5_file.create_dataset("x_test", (test_size, 512), np.float32)
    hdf5_file.create_dataset("y_test", (test_size,), np.int32)
    
    index = 0
    for _, data in enumerate(hb_train_loader_bln_3, 0):
        inputs, labels = cuda(data)
        outputs = res_net(inputs.expand(-1, 3, -1, -1))
        hdf5_file["x_train"][index:index + outputs.shape[0], ...] = outputs.cpu().numpy()
        hdf5_file["y_train"][index:index + outputs.shape[0]] = labels.cpu().numpy()
        index += outputs.shape[0]
    print(index)
    index = 0
    for _, data in enumerate(hb_test_loader_bln_3, 0):
        inputs, labels = cuda(data)
        outputs = res_net(inputs.expand(-1, 3, -1, -1))
        hdf5_file["x_test"][index:index + outputs.shape[0], ...] = outputs.cpu().numpy()
        hdf5_file["y_test"][index:index + outputs.shape[0]] = labels.cpu().numpy()
        index += outputs.shape[0]

152471


In [8]:
with h5py.File('hb_rp_bih_intermediate.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (train_size, 512), np.float32)
    hdf5_file.create_dataset("y_train", (train_size,), np.int32)
    hdf5_file.create_dataset("x_test", (test_size, 512), np.float32)
    hdf5_file.create_dataset("y_test", (test_size,), np.int32)
    
    index = 0
    for _, data in enumerate(hb_train_loader_bln_4, 0):
        inputs, labels = cuda(data)
        outputs = res_net(inputs.expand(-1, 3, -1, -1))
        hdf5_file["x_train"][index:index + outputs.shape[0], ...] = outputs.cpu().numpy()
        hdf5_file["y_train"][index:index + outputs.shape[0]] = labels.cpu().numpy()
        index += outputs.shape[0]
    print(index)
    index = 0
    for _, data in enumerate(hb_test_loader_bln_4, 0):
        inputs, labels = cuda(data)
        outputs = res_net(inputs.expand(-1, 3, -1, -1))
        hdf5_file["x_test"][index:index + outputs.shape[0], ...] = outputs.cpu().numpy()
        hdf5_file["y_test"][index:index + outputs.shape[0]] = labels.cpu().numpy()
        index += outputs.shape[0]
    print(index)

152471
21892


In [11]:
with h5py.File('hb_gaf_rp_bih_intermediate.hdf5', mode='w') as hdf5_file:
    hdf5_file.create_dataset("x_train", (train_size, 1024), np.float32)
    hdf5_file.create_dataset("y_train", (train_size,), np.int32)
    hdf5_file.create_dataset("x_test", (test_size, 1024), np.float32)
    hdf5_file.create_dataset("y_test", (test_size,), np.int32)
    
    index = 0
    for _, (data_gaf, data_rp) in enumerate(zip(hb_train_loader_bln_3, hb_train_loader_bln_4), 0):
        inputs_gaf, labels = cuda(data_gaf)
        inputs_rp, _ = cuda(data_rp)
        outputs_gaf = res_net_gaf(inputs_gaf.expand(-1, 3, -1, -1))
        outputs_rp = res_net_rp(inputs_rp.expand(-1, 3, -1, -1))
        hdf5_file["x_train"][index:index + outputs_gaf.shape[0], :512] = outputs_gaf.cpu().numpy()
        hdf5_file["x_train"][index:index + outputs_rp.shape[0], 512:1024] = outputs_rp.cpu().numpy()
        hdf5_file["y_train"][index:index + outputs_gaf.shape[0]] = labels.cpu().numpy()
        index += outputs_gaf.shape[0]
    print(index)
    index = 0
    for _, (data_gaf, data_rp) in enumerate(zip(hb_test_loader_bln_3, hb_test_loader_bln_4), 0):
        inputs_gaf, labels = cuda(data_gaf)
        inputs_rp, _ = cuda(data_rp)
        outputs_gaf = res_net_gaf(inputs_gaf.expand(-1, 3, -1, -1))
        outputs_rp = res_net_rp(inputs_rp.expand(-1, 3, -1, -1))
        hdf5_file["x_test"][index:index + outputs_gaf.shape[0], :512] = outputs_gaf.cpu().numpy()
        hdf5_file["x_test"][index:index + outputs_rp.shape[0], 512:1024] = outputs_rp.cpu().numpy()
        hdf5_file["y_test"][index:index + outputs_gaf.shape[0]] = labels.cpu().numpy()
        index += outputs_gaf.shape[0]

152471


In [7]:
net = nn.Sequential(
                nn.Linear(in_features=1024, out_features=512, bias=False),
                nn.ReLU(),
#                 nn.Dropout(p=0.5),
                nn.Linear(in_features=512, out_features=256, bias=False),
                nn.ReLU(),
                nn.Linear(in_features=256, out_features=128, bias=True),
                nn.ReLU(),
#                 nn.Dropout(p=0.5),
                nn.Linear(in_features=128, out_features=5, bias=True))
net.cuda()

Sequential(
  (0): Linear(in_features=1024, out_features=512, bias=False)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=False)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=5, bias=True)
)

In [8]:
# Custom class for defining dataset for training with augmentation
class DatasetFusion(Dataset):

    def __init__(self, path, data_type):
        """ Intialize the dataset
        """
        self.path = path
        self.file = h5py.File(path, 'r')
        self.images = self.file['x_{}'.format(data_type)]
        self.labels = self.file['y_{}'.format(data_type)]
                
        self.len = self.images.shape[0]
        if data_type == 'train':
            # no augmentation as we are transforming heartbeats to images via gramian angular field
            self.transform = transforms.Compose([

                                              transforms.ToTensor()])
        else:
            self.transform = transforms.Compose([transforms.ToTensor()])

    # You must override __getitem__ and __len__
    def __getitem__(self, index):
        """ Get a sample from the dataset
        """
        # unsqueeze adds dimension to image -> converts to 1x224x224 since we don't have rgb
        return torch.tensor(self.images[index].astype('float32')), \
                torch.tensor(self.labels[index], dtype=torch.long)

    def __len__(self):
        """
        Total number of samples in the dataset
        """
        return self.len

In [9]:
hb_train_loader = torch.utils.data.DataLoader(DatasetFusion('/home/asif/heartbeat/hb_gaf_rp_bih_intermediate.hdf5', 'train'), 
                                                batch_size=512, shuffle=True)
hb_test_loader = torch.utils.data.DataLoader(DatasetFusion('/home/asif/heartbeat/hb_gaf_rp_bih_intermediate.hdf5', 'test'), 
                                                batch_size=512, shuffle=False)

In [10]:
class_weights = cuda(torch.tensor([1.0,2.0, 1.0, 2.0, 1.0]))
criterion = nn.CrossEntropyLoss(weight=class_weights)
# criterion = nn.CrossEntropyLoss()

In [11]:
optimizer = torch.optim.Adam([
                                          {"params": net[0].parameters(), "lr": 0.001},
                                          {"params": net[2].parameters(), "lr": 0.001},
                                          {"params": net[4].parameters(), "lr": 0.002},
#                                           {"params": net[6].parameters(), "lr": 0.002},
                                           ],
                                lr=0.001, betas=(0.5, 0.999))

In [69]:
optimizer_res_net = torch.optim.Adam([
                                          {"params": res_net_gaf.fc[0].parameters(), "lr": 0.001},
                                          {"params": res_net_gaf.fc[2].parameters(), "lr": 0.001},
                                          {"params": res_net_gaf.fc[4].parameters(), "lr": 0.001},
                                           ],  
                                lr=0.0001, betas=(0.9, 0.999))

In [12]:
def train(net, train_loader, criterion, optimizer, test_loader, num_epochs=25):
    net.train()
    train_acc_max = 0
    test_acc_max = 0
    recall_max = 0
    f1_max = 0

#     scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma = 0.95)
    
    
    for epoch in range(num_epochs):  # loop over the dataset multiple times
        
        net.train()

        total = 0
        correct = 0
   
        running_loss = 0.0
    
        for i, data in enumerate(train_loader, 0):
            
            # get the inputs; data is a list of [inputs, labels]
            
            inputs, labels = cuda(data)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs) # already in RGB
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
        
#         scheduler.step()
        
        print('End of epoch {}, Loss {}'.format(epoch + 1, running_loss / len(train_loader)))
        
        train_acc = correct / total
        print('Train accuracy: {}'.format(train_acc))
        test_acc, all_true, all_pred = test(net, test_loader)
        print('Test accuracy: {}'.format(test_acc))
        print(classification_report(all_true, all_pred, target_names=['N', 'S', 'V', 'F', 'Q']))
        recall = recall_score(all_true, all_pred, average='macro')
        f1 = f1_score(all_true, all_pred, average='macro')
        
        # Saving best checkpoint based on performance on test data            
        if recall > recall_max:
            recall_max = recall
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, recall, f1, 'fc_net_fusion', 'test_recall')
            
        if f1 > f1_max:
            f1_max = f1
            save_checkpoint(epoch + 1, net, optimizer, train_acc, test_acc, recall, f1, 'fc_net_fusion', 'test_f1')

    print('Finished Training')
    
def test(net, test_loader):
    net.eval()
    correct = 0
    total = 0
    all_true = []
    all_pred = []
    with torch.no_grad():
        for i, data in enumerate(test_loader, 0):
            inputs, labels = cuda(data)
            all_true.extend(labels.cpu().tolist())
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            all_pred.extend(predicted.cpu().tolist())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = correct / total
#     print('Accuracy of the network on the images: %d %%' % (100 * acc))
    return acc, all_true, all_pred

def save_checkpoint(epoch, net, optimizer, train_acc, test_acc, recall, f1, net_name, param):
    checkpoint = {
        'net': net.state_dict(),
        'optimizer': optimizer.state_dict(),
        'train_acc': train_acc,
        'test_acc': test_acc,
        'recall': recall,
        'f1': f1
    }
    torch.save(checkpoint, '{}_{}_best.chk'.format(net_name, param))

In [13]:
train(net, hb_train_loader, criterion, optimizer, hb_test_loader, 30)

End of epoch 1, Loss 0.764226275652447
Train accuracy: 0.7051570462579769
Test accuracy: 0.8439612643888178
              precision    recall  f1-score   support

           N       0.92      0.94      0.93     18118
           S       0.17      0.41      0.24       556
           V       0.82      0.39      0.53      1448
           F       0.16      0.57      0.25       162
           Q       0.76      0.31      0.44      1608

    accuracy                           0.84     21892
   macro avg       0.56      0.52      0.48     21892
weighted avg       0.88      0.84      0.85     21892

End of epoch 2, Loss 0.4451605366955467
Train accuracy: 0.8518210020266149
Test accuracy: 0.1346153846153846
              precision    recall  f1-score   support

           N       0.99      0.12      0.22     18118
           S       0.03      1.00      0.06       556
           V       0.86      0.04      0.07      1448
           F       0.77      0.17      0.27       162
           Q       0.43

/home/asif/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


End of epoch 14, Loss 0.24712461819434725
Train accuracy: 0.9104747788103967
Test accuracy: 0.9710853279736891
              precision    recall  f1-score   support

           N       0.99      0.99      0.99     18118
           S       0.75      0.83      0.79       556
           V       0.85      0.93      0.89      1448
           F       0.76      0.58      0.66       162
           Q       0.95      0.87      0.91      1608

    accuracy                           0.97     21892
   macro avg       0.86      0.84      0.85     21892
weighted avg       0.97      0.97      0.97     21892

End of epoch 15, Loss 0.271583838032046
Train accuracy: 0.9229164890372595
Test accuracy: 0.9727297642974603
              precision    recall  f1-score   support

           N       0.99      1.00      0.99     18118
           S       0.78      0.77      0.77       556
           V       0.92      0.87      0.89      1448
           F       0.75      0.66      0.70       162
           Q       0

End of epoch 28, Loss 0.01696021671113152
Train accuracy: 0.9940316519206931
Test accuracy: 0.9862050063950302
              precision    recall  f1-score   support

           N       1.00      1.00      1.00     18118
           S       0.94      0.81      0.87       556
           V       0.92      0.96      0.94      1448
           F       0.85      0.78      0.81       162
           Q       0.97      0.97      0.97      1608

    accuracy                           0.99     21892
   macro avg       0.94      0.90      0.92     21892
weighted avg       0.99      0.99      0.99     21892

End of epoch 29, Loss 0.014443835595953934
Train accuracy: 0.9945891349830459
Test accuracy: 0.9809976247030879
              precision    recall  f1-score   support

           N       0.99      1.00      1.00     18118
           S       0.92      0.76      0.84       556
           V       0.90      0.93      0.92      1448
           F       0.54      0.79      0.64       162
           Q     

In [87]:
net(b)[0]

tensor([-0.1094, -0.0180,  0.0353, -0.0687,  0.0714], grad_fn=<SelectBackward>)

In [47]:
np.expand_dims(a, 0)

array([[1],
       [2],
       [3]])

In [49]:
torch.tensor(np.array([1,2, 3])).shape

torch.Size([3])